In [1]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import itertools
import random
import time
#import sage

In [4]:
A = np.array([[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0],
              [1,2,3,4,0,1,0,0,0,0,0,0,0,0,0],
                [2,3,4,5,0,0,1,0,0,0,0,0,0,0,0],
             [2,3,4,5,0,0,0,1,0,0,0,0,0,0,0],
             [2,3,4,5,0,0,0,0,1,0,0,0,0,0,0],
             [2,3,4,5,0,0,0,0,0,1,0,0,0,0,0],
             [2,3,4,5,0,0,0,0,0,0,1,0,0,0,0],
             [2,3,4,5,0,0,0,0,0,0,0,1,0,0,0],
             [2,3,4,5,0,0,0,0,0,0,0,0,1,0,0],
             [2,3,4,5,0,0,0,0,0,0,0,0,0,1,0],
             [2,3,4,5,0,0,0,0,0,0,0,0,0,0,1]])
c = np.array([-2,-3,-4,-5,-6,-7,-8,0,0,0,0,0,0,0,0])
Matrix(A)

Matrix([
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, 2, 3, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

# Pre

In [5]:
def to_polynomial(coef,vars):
            '''
            Function to define a single row of the coefficient as a polynomial
            '''
            res1 = 1
            res2 = 1
            for i in range(len(coef)):
                if coef[i] > 0:
                    res1 = res1*vars[i]**coef[i]
                elif coef[i] < 0:
                    res2 = res2*vars[i]**(-coef[i])
            res = res1 - res2
            return res

In [6]:
def monomial(p):
    return [prod(x**k for x, k in zip(p.gens, mon)) for mon in p.monoms()]

In [7]:
# Define symbolic variables xs for each column (index 0 in Python)
sym_str_x = 'x:' + str(A.shape[1])
xs = symbols(sym_str_x)

# Main

In [8]:
def gen(A):
    # column/row dimension of a matrix
    r = A.shape[0]
    n = A.shape[1]
    A = np.array(A)
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)
    
    # Calculate a basis for the kernel
    # http://www.numbertheory.org/php/mlll.html
    # the website can fix the gap
    def Ker(A):
        A = Matrix(A)
        B = np.array(Matrix(A).nullspace()).transpose()
        C = Matrix(B[0]).transpose()
        return C
    
    # find an equivalent basis with all base vectors lying in the same orthant
    def same_orthant(A):    
        # compute the highest value in abstract value in a numpy natrix
        def abs_max(A):
            A = np.array(A)
            return abs(max(A.min(), A.max(), key=abs))

        for i in range(1,A.shape[0]):
            A[i,:] = A[i,:]+A[i-1,:]*(1+abs_max(A[i,:]))
        A_fin = np.array(A)
        A_fin = A_fin.astype(int)
        A_fin = Matrix(A_fin)
        return A_fin
    
    # compute J and prepare the function phi & phi_inv
    def nega_index(A):
        J = [ A[A.shape[0]-1,k]<0 for k in range(0,A.shape[1])]
        return J

    def reverse_sign(A):
        n = A.shape[1]
        A = np.array(A)
        for i in range(0,n):
            if nega_index(A)[i]:
                A[:,i] = - A[:,i]
        A_fin = A.astype(int)
        A_fin = Matrix(A_fin)
        return A_fin

    def phi(A):
        G_J=[]
        for k in range(0,A.shape[0]):
            G_J.append(to_polynomial(A[k,:],xs))
        return G_J

    def phi_inv(G_J):
        G_J_len = len(G_J)

        temp = []
        vp = [0]*n
        vm = [0]*n
        for k in range(0,G_J_len):
            for i in range(0,n):
                p = monomial(Poly(G_J[k]))[0]
                m = monomial(Poly(G_J[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            temp.append(v)
            temp_fin = Matrix(temp)
        return temp_fin, temp
    
    
    def TJ_operation(G_J,J):
        for j in J:
            if j:
                # change position of xs[j]
                xs_list = list(xs)
                index = J.index(j)
                J[index] = False
                temp = xs_list[index]
                xs_list.remove(temp)
                xs_list.insert(0,temp)

                G_J = list(groebner(G_J, xs_list , order='lex'))

                A3, _ = phi_inv(G_J)
                A3[:,index] = - A3[:,index]
                G_J = phi(A3)
            print('Done for once!')

        return G_J
    
   
    # MAIN
    # generating set for the corresponding matrix kernel
    A1 = Ker(A)

    # matrix pre-operation
    A2 = same_orthant(A1)
    J = nega_index(A2)
    A3 = reverse_sign(A2)
    
    # groebner basis T_J operation
    G_J = phi(A3)
    G_zero = TJ_operation(G_J,J)
    return G_zero

In [9]:
def groeb(G_zero,c):
    # column/row dimension of a matrix
    n = c.shape[0]
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)

    def phi_inv(G_J):
        G_J_len = len(G_J)

        temp = []
        vp = [0]*n
        vm = [0]*n
        for k in range(0,G_J_len):
            for i in range(0,n):
                p = monomial(Poly(G_J[k]))[0]
                m = monomial(Poly(G_J[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            temp.append(v)
            temp_fin = Matrix(temp)
        return temp_fin, temp

    # define new order in Sympy
    class compositeOrder(polys.orderings.MonomialOrder):
        """Composite-Lexicographic order of monomials. """
        alias = 'clex'
        is_global = True

        def __call__(self, monomial):
            return ( np.dot(c,np.array(monomial)),sum(monomial), monomial )
            #return ( monomial )
    polys.orderings.__all__.append('clex')
    clex = compositeOrder()
    polys.orderings._monomial_key['clex'] = compositeOrder() 
    
    # apply groebner basis algorithm 
    G_fin = groebner(G_zero,xs,order='clex')
    _ , G_fin = phi_inv(list(G_fin))
    return G_fin

# Experiment I

In [10]:
def augmentation(z_feas,c,T):
    # z_feaas: feasible point ; c: cost; T: universal test set
    exist_aug = True
    i = 0
    while exist_aug:
        exist_aug = False
        for t in T:
            if np.dot(c, t, out=None)>0 and np. all((z_feas-t>=0)):
                z_feas = z_feas-t
                i = 1+i
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
            if np.dot(c, t, out=None)<0 and np. all((z_feas+t>=0)):
                z_feas = z_feas+t
                i = i+1
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
    #print('Achieve an optimum!')
    return z_feas

In [11]:
def feasible_solu(b):
    return [0,0,0,0,0,0,0,b[0],b[1],b[2],b[3],b[4],b[5],b[6],b[7]]

In [12]:
#def feasible_solu(b):
#    return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,b[0],b[1],b[2],b[3],b[4],b[5],b[6],b[7],b[8],b[9]]

In [13]:
start_time = time.time()

T_1 = gen(A)

print("--- %s seconds ---" % (time.time() - start_time))

Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
Done for once!
--- 0.5166022777557373 seconds ---


In [14]:
# loadPackage "Quasidegrees"
# A=matrix{{1,1,1,1,1,1},{1,2,1,2,3,0},{0,2,2,0,1,1}}
# R=QQ[a..f]
# toricIdeal(A,R)

#T_2 

In [15]:
start_time = time.time()

opt = []
for i in range(0,600,3):
    b = [300+i,300+i,200+10*i,200+10*i,200+10*i,200+10*i,200+10*i,200+10*i]
    temp = augmentation(feasible_solu(b),c,groeb(T_1,c))
    opt.append(temp)
    print(i)

print("--- %s seconds ---" % (time.time() - start_time))

0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297
300
303
306
309
312
315
318
321
324
327
330
333
336
339
342
345
348
351
354
357
360
363
366
369
372
375
378
381
384
387
390
393
396
399
402
405
408
411
414
417
420
423
426
429
432
435
438
441
444
447
450
453
456
459
462
465
468
471
474
477
480
483
486
489
492
495
498
501
504
507
510
513
516
519
522
525
528
531
534
537
540
543
546
549
552
555
558
561
564
567
570
573
576
579
582
585
588
591
594
597
--- 17.311414003372192 seconds ---


In [ ]:
#opt